made this to try to solve the memory issues of the last notebook - intention is just to reload the models, and save the tp, fp, and auc for each. the data processing took up a ton of memory to had to do this in stages and refresh every so often. then plotted stuff in week5_roc

In [1]:
import uproot as ur
import awkward as ak
import numpy as np

data_path = '/fast_scratch/atlas_images/v01-45/'

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

# energyflow imports
import energyflow as ef
from energyflow.archs import PFN
from energyflow.utils import data_split

from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append('/Users/swiatlow/Code/ML4P/LCStudies')
sys.path.append('/home/mswiatlowski/start_tf/LCStudies')
import graph_util as gu
import plot_util as pu

In [3]:
#first: data processing: (careful she takes a long time)
path_pipm = data_path + 'pipm_medium.root'
path_pi0  = data_path + 'pi0_medium.root'

tree_pipm = ur.open(path_pipm)['EventTree']
tree_pi0  = ur.open(path_pi0)['EventTree']

geoDict_tree_pipm = ur.open(path_pipm)['CellGeo']
geoDict_tree_pi0  = ur.open(path_pi0)["CellGeo"]

cell_id_pipm = gu.loadArrayBranchFlat('cluster_cell_ID', tree_pipm, 2000)
cell_e_pipm = gu.loadArrayBranchFlat('cluster_cell_E', tree_pipm, 2000)
cell_id_pi0 = gu.loadArrayBranchFlat('cluster_cell_ID', tree_pi0, 2000)
cell_e_pi0 = gu.loadArrayBranchFlat('cluster_cell_E', tree_pi0, 2000)

geoDict_pipm = gu.loadGraphDictionary(geoDict_tree_pipm)
geoDict_pi0  = gu.loadGraphDictionary(geoDict_tree_pi0)

cell_phi_pipm = gu.convertIDToGeo(cell_id_pipm, 'cell_geo_phi', geoDict_pipm)
cell_eta_pipm = gu.convertIDToGeo(cell_id_pipm, 'cell_geo_eta', geoDict_pipm)
cell_phi_pi0 = gu.convertIDToGeo(cell_id_pi0, 'cell_geo_phi', geoDict_pi0)
cell_eta_pi0 = gu.convertIDToGeo(cell_id_pi0, 'cell_geo_eta', geoDict_pi0)

clus_eta_pipm = gu.loadVectorBranchFlat('cluster_Eta', tree_pipm)
clus_E_pipm = gu.loadVectorBranchFlat('cluster_E', tree_pipm)

clus_eta_pi0 = gu.loadVectorBranchFlat('cluster_Eta', tree_pi0)
clus_E_pi0 = gu.loadVectorBranchFlat('cluster_E', tree_pi0)

clus_phi_pipm = gu.loadVectorBranchFlat('cluster_Phi', tree_pipm)
clus_phi_pi0  = gu.loadVectorBranchFlat('cluster_Phi', tree_pi0)

In [4]:
#HERE: change the eta cuts!
eta_mask_pipm = (abs(clus_eta_pipm) > 1.4) | (abs(clus_eta_pipm) < 2.5)
eta_mask_pi0  =(abs(clus_eta_pi0) > 1.4) | (abs(clus_eta_pi0) < 2.5)

energy_mask_pipm = clus_E_pipm > 0.5
energy_mask_pi0 = clus_E_pi0 > 0.5

selection_pipm = eta_mask_pipm & energy_mask_pipm
selection_pi0 = eta_mask_pi0 & energy_mask_pi0

In [5]:
#dont rerun
cell_eta_norm_pipm = np.nan_to_num(cell_eta_pipm - clus_eta_pipm[:, None])
cell_eta_norm_pi0 = np.nan_to_num(cell_eta_pi0 - clus_eta_pi0[:, None])
cell_phi_norm_pipm = np.nan_to_num(cell_phi_pipm - clus_phi_pipm[:, None])
cell_phi_norm_pi0 = np.nan_to_num(cell_phi_pi0 - clus_phi_pi0[:, None])

cell_e_norm_pipm = np.nan_to_num(np.log(cell_e_pipm), posinf = 0, neginf=0)
cell_e_norm_pi0 = np.nan_to_num(np.log(cell_e_pi0), posinf = 0, neginf=0)

# Working after some careful manipulation of the masks
cell_samp_pipm = gu.convertIDToGeo(cell_id_pipm, 'cell_geo_sampling', geoDict_pipm)
cell_samp_pi0 = gu.convertIDToGeo(cell_id_pi0, 'cell_geo_sampling', geoDict_pi0)

#normalize to a lower value fairly arbitrarily
cell_samp_norm_pipm = cell_samp_pipm * 0.1
cell_samp_norm_pi0 = cell_samp_pi0 * 0.1


<ipython-input-5-9ca06993e40d>:7: RuntimeWarning: divide by zero encountered in log
  cell_e_norm_pipm = np.nan_to_num(np.log(cell_e_pipm), posinf = 0, neginf=0)
<ipython-input-5-9ca06993e40d>:8: RuntimeWarning: divide by zero encountered in log
  cell_e_norm_pi0 = np.nan_to_num(np.log(cell_e_pi0), posinf = 0, neginf=0)


In [6]:
#rerun!!!
X_pipm = np.stack((cell_e_norm_pipm[selection_pipm], cell_eta_norm_pipm[selection_pipm], cell_phi_norm_pipm[selection_pipm], cell_samp_norm_pipm[selection_pipm]), axis = 2)
X_pi0  = np.stack((cell_e_norm_pi0[selection_pi0], cell_eta_norm_pi0[selection_pi0], cell_phi_norm_pi0[selection_pi0], cell_samp_norm_pi0[selection_pi0]), axis = 2)

len(X_pipm), len(X_pi0)

(592857, 395440)

In [7]:
X = np.concatenate((X_pipm[:90000], X_pi0[:90000]))
Y = tf.keras.utils.to_categorical(np.concatenate((np.ones(90000), np.zeros(90000)))) #one hot
print(len(X), len(Y))

180000 180000


In [8]:
def compiled_rocs(X, Y, model, test_size, outfile):
    (X_train, X_val, X_test, Y_train, Y_val, Y_test) = data_split(X, Y, val=10000, test=test_size)

    preds = model.predict(X_test, batch_size=1000)
    pfn_fp, pfn_tp, threshs = roc_curve(Y_test[:,1], preds[:,1])
    # get area under the ROC curve
    auc = roc_auc_score(Y_test[:,1], preds[:,1])
    np.savez(outfile, pfn_fp, pfn_tp, auc)
    return auc #instead of doing this I should definitely just return these when making the model
                               #so defs make that change once they improve models/data/etc !!

In [10]:
models_b2531 = tf.keras.models.load_model('./pfnmodel_b2531.h5')

In [11]:
compiled_rocs(X, Y, models_b2531, 40000, 'roc_b2531')

0.9822372141408154

In [15]:
models_l05 = tf.keras.models.load_model('./pfnmodel_l05.h5')

In [16]:
compiled_rocs(X, Y, models_l05, 20000, 'roc_l05')

0.9821431593013783

In [17]:
models_l07 = tf.keras.models.load_model('./pfnmodel_l07.h5')

In [21]:
compiled_rocs(X, Y, models_l07, 40000, 'roc_l07')

0.9793627303601405

In [9]:
models_b0715 = tf.keras.models.load_model('./pfnmodel_b0715.h5')

In [10]:
compiled_rocs(X,Y, models_b0715, 40000, 'roc_b0715')

0.9668667722410309

In [11]:
models_g15 = tf.keras.models.load_model('./pfnmodel_g15.h5')

In [15]:
compiled_rocs(X, Y, models_g15, 40000, 'roc_g15')

0.9551590465105598

In [16]:
models_l15 = tf.keras.models.load_model('./pfnmodel_l15.h5')

In [20]:
compiled_rocs(X,Y, models_l15, 40000, 'roc_l15')

0.9713802628694845

In [21]:
models_l1 = tf.keras.models.load_model('./pfnmodel_l1.h5')

In [25]:
compiled_rocs(X, Y, models_l1, 40000, 'roc_l1')

0.9800122575943051

In [26]:
models_b1425 = tf.keras.models.load_model('./pfnmodel_b1425.h5')

In [30]:
compiled_rocs(X, Y, models_b1425, 40000, 'roc_b1425')

0.9429751786336393

In [9]:
models_all = tf.keras.models.load_model('./pfnmodel_gall.h5')

In [10]:
compiled_rocs(X,Y, models_all, 40000, 'roc_all')

0.9613265061124728